# 2048


In [1]:
import numpy as np
import random as rand

class Env:
    def __init__(self, size):
        self.size = size
        self.bord = np.zeros((self.size,self.size))
        self.score = 0
        self.done = False
        self.start()
    
    def findEmpty(self):
        # 빈칸을 확인해서 좌표 return / 없으면 None return
        z_list = np.array([[r,c] for r,c in zip(np.where(self.bord==0)[0], np.where(self.bord==0)[1])])
        if z_list.size == 0:
            return None
        location = rand.choice(z_list)
        return location

    def isfull(self):
        # bord가 숫자로 가득 차있는경우 True/ 빈칸이 있는경우 False 반환
        if 0 in self.bord.reshape(-1):
            return False
        else:
            return True
        
    def mkblock(self):
        # 빈칸에 확률에 맞춰서 2또는 4 블럭을 생성
        location = self.findEmpty()
        self.bord[location[0]][location[1]] = 2 if rand.random() < 0.9 else 4
        self.score += self.bord[location[0]][location[1]]
    
    def merge(self, lstin):
        # 합치는 연산
        lst = lstin
        if lst.size != len(set(lst)):
                    for t in range(lst.size-1):
                        if lst[t] == lst[t+1]:
                            lst[t] *= 2
                            lst[t+1] = 0
                    lst = lst[np.where(lst != 0)]
        return lst
    
    def move(self, index):
        # index에 따라 움직임
        # 움직임이 가능한지 확인하고
        if self.canMove():
            # 가능하다면 움직임 (0 - 상) (1 - 하) (2 - 좌) (3 - 우)
            if index == 0:
                for c in range(self.size):
                    col = self.bord[:,c][np.where(self.bord[:,c] != 0)]
                    col = self.merge(col)
                    pd = 4-col.size
                    if np.any(self.bord[:,c] != np.pad(col,(0,pd))):
                        self.bord[:,c] = np.pad(col,(0,pd))
            elif index == 1:
                for c in range(self.size):
                    col = self.bord[:,c][np.where(self.bord[:,c] != 0)]
                    col = self.merge(col[::-1])[::-1]
                    pd = 4-col.size
                    if np.any(self.bord[:,c] != np.pad(col,(pd, 0))):
                        self.bord[:,c] = np.pad(col,(pd, 0))
            elif index == 2:
                for r in range(self.size):
                    row = self.bord[r,:][np.where(self.bord[r,:] != 0)]
                    row = self.merge(row)
                    pd = 4-row.size
                    if np.any(self.bord[r,:] != np.pad(row,(0, pd))):
                        self.bord[r,:] = np.pad(row,(0, pd))
            elif index == 3:
                for r in range(self.size):
                    row = self.bord[r,:][np.where(self.bord[r,:] != 0)]
                    row = self.merge(row[::-1])[::-1]
                    pd = 4-row.size
                    if np.any(self.bord[r,:] != np.pad(row,(pd, 0))):
                        self.bord[r,:] = np.pad(row,(pd, 0))
            # 움직인 다음에는 무조건 공간이 생기므로 새로운 블럭 생성
            self.mkblock()
                
        else:
            # 움직일 수 없는 경우는 게임종료
            self.gameOver()
        
            
    def canMove(self):
        # 움직임이 가능한지 확인 움직임이 가능하면 ture
        # 반환 불가능하면 게임 종료 
        for i in range(self.size):
            for j in range(self.size - 1):
                if self.bord[i][j] == self.bord[i][j + 1]:
                    return True
        for j in range(self.size):
            for i in range(self.size - 1):
                if self.bord[i][j] == self.bord[i + 1][j]:
                    return True
        if not self.isfull():
            return True
        self.done = True
        self.show()
        return False 

    def gameOver(self):
        # done 게임 종료 선언
        self.done = True
        print('Game Over!!')
        
    def show(self):
        # bord와 score 출력
        print(self.bord)
        print('Score : ',self.score)
        
    def start(self):
        # 게임 시작
        i = 0
        self.mkblock()
        self.mkblock()
        self.show()
        while(not self.done):
            # 임의의 이동 선택중
            k = rand.choice([0,1,2,3])
            self.move(k)
            i += 1
            if i % 50 == 0:
                self.show()
        

In [2]:
env = Env(4)

[[0. 2. 2. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Score :  4.0
[[ 8.  2.  0.  0.]
 [ 2.  4. 32.  8.]
 [32.  4.  0.  0.]
 [ 4.  0.  0.  0.]]
Score :  96.0
[[ 8.  4.  4.  0.]
 [16.  8. 32. 64.]
 [ 2. 32.  4.  0.]
 [ 4.  4.  0.  2.]]
Score :  184.0
[[  8.   8.   4.   2.]
 [ 32.   8.  32.   2.]
 [128.  32.   4.   2.]
 [  2.  16.   0.   0.]]
Score :  280.0
[[  4.   4.   8.   4.]
 [ 16. 256.  16.  16.]
 [  0.   4.  32.   4.]
 [  2.   8.   2.   0.]]
Score :  376.0
[[  8.   2.   8.   4.]
 [  2.   4.  64.   2.]
 [  4.  32. 256.   8.]
 [  2.  16.   8.  32.]]
Score :  452.0
Game Over!!


In [4]:
from tkinter import *

In [5]:
SIZE = 500
GRID_LEN = 4
GRID_PADDING = 10

BACKGROUND_COLOR_GAME = "#92877d"
BACKGROUND_COLOR_CELL_EMPTY = "#9e948a"
BACKGROUND_COLOR_DICT = {   2:"#eee4da", 4:"#ede0c8", 8:"#f2b179", 16:"#f59563", \
                            32:"#f67c5f", 64:"#f65e3b", 128:"#edcf72", 256:"#edcc61", \
                            512:"#edc850", 1024:"#edc53f", 2048:"#edc22e" }

CELL_COLOR_DICT = { 2:"#776e65", 4:"#776e65", 8:"#f9f6f2", 16:"#f9f6f2", \
                    32:"#f9f6f2", 64:"#f9f6f2", 128:"#f9f6f2", 256:"#f9f6f2", \
                    512:"#f9f6f2", 1024:"#f9f6f2", 2048:"#f9f6f2" }

FONT = ("Verdana", 40, "bold")

class GameGrid(Frame):
    def __init__(self):
        Frame.__init__(self)

        self.grid()
        self.master.title('2048')

        self.grid_cells = []
        self.init_grid()
        self.init_matrix()
        self.update_grid_cells()
        
        self.wait_visibility()
        self.after(10,self.make_move)
        self.env = Env(GRID_LEN)
        
    def init_grid(self):
        background = Frame(self, bg=BACKGROUND_COLOR_GAME, width=SIZE, height=SIZE)
        background.grid()
        for i in range(GRID_LEN):
            grid_row = []
            for j in range(GRID_LEN):
                cell = Frame(background, bg=BACKGROUND_COLOR_CELL_EMPTY, width=SIZE/GRID_LEN, height=SIZE/GRID_LEN)
                cell.grid(row=i, column=j, padx=GRID_PADDING, pady=GRID_PADDING)
                # font = Font(size=FONT_SIZE, family=FONT_FAMILY, weight=FONT_WEIGHT)
                t = Label(master=cell, text="", bg=BACKGROUND_COLOR_CELL_EMPTY, justify=CENTER, font=FONT, width=4, height=2)
                t.grid()
                grid_row.append(t)

            self.grid_cells.append(grid_row)

    def gen(self):
        return randint(0, GRID_LEN - 1)

    def init_matrix(self):
        self.matrix = self.env.

        self.matrix=add_two(self.matrix)
        self.matrix=add_two(self.matrix)

    def update_grid_cells(self):
        for i in range(GRID_LEN):
            for j in range(GRID_LEN):
                new_number = self.matrix[i][j]
                if new_number == 0:
                    self.grid_cells[i][j].configure(text="", bg=BACKGROUND_COLOR_CELL_EMPTY)
                else:
                    self.grid_cells[i][j].configure(text=str(new_number), bg=BACKGROUND_COLOR_DICT[new_number], fg=CELL_COLOR_DICT[new_number])
        self.update_idletasks()
        
    def make_move(self):
        output = learned_sess.run([single_output],feed_dict = {single_dataset:change_values(self.matrix)})
        move = np.argmax(output[0])
        self.matrix,done,tempo = controls[move](self.matrix)
        done=True
        
        #if game_state(self.matrix)=='win':
        #    self.grid_cells[1][1].configure(text="You",bg=BACKGROUND_COLOR_CELL_EMPTY)
        #    self.grid_cells[1][2].configure(text="Win!",bg=BACKGROUND_COLOR_CELL_EMPTY)
        #    done=False
        if game_state(self.matrix)=='lose':
            self.grid_cells[1][1].configure(text="You",bg=BACKGROUND_COLOR_CELL_EMPTY)
            self.grid_cells[1][2].configure(text="Lose!",bg=BACKGROUND_COLOR_CELL_EMPTY)
            done=False
            
        self.matrix = add_two(self.matrix)
        self.update_grid_cells()
        
        
        if(done==True):
            self.after(7,self.make_move)
        else:
            time.sleep(3)
            self.init_matrix()
            self.update_grid_cells()
            self.after(7,self.make_move)

    def generate_next(self):
        empty_cells = []
        for i in range(len(mat)):
            for j in range(len(mat)):
                if(mat[i][j]==0):
                    empty_cells.append((i,j))
        if(len(empty_cells)==0):
            return 0,false
        index_pair = empty_cells[random.randint(0,len(empty_cells)-1)]
        index = index_pair
        self.matrix[index[0]][index[1]] = 2

root = Tk()
gamegrid = GameGrid()
root.mainloop()

NameError: name 'new_game' is not defined